In [ ]:
import pandas as pd
import os

In [2]:
path = './raw/resale-flat-prices'

In [3]:
dfs = []
for file in os.listdir(path):
    if file.endswith('.csv'):
        print(file)
        file_path = os.path.join(path, file)
        temp = pd.read_csv(file_path)
        temp = temp.drop(['remaining_lease'], axis=1, errors='ignore')
        print(temp.shape)
        dfs.append(temp)

resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv
(52203, 10)
resale-flat-prices-based-on-approval-date-1990-1999.csv
(287200, 10)
resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv
(97043, 10)
resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
(369651, 10)
resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv
(37153, 10)


In [4]:
full = pd.concat(dfs,axis=0)

In [18]:
full['psm'] = full['resale_price']/full['floor_area_sqm']
full['flat_type'] = full['flat_type'].str.lower()
full['flat_model'] = full['flat_model'].str.lower()
full['year'] = full['month'].apply(lambda x: x.split('-')[0]).astype(int)

In [6]:
full.flat_type.unique()

array(['2 room', '3 room', '4 room', '5 room', 'executive',
       'multi-generation', '1 room', 'multi generation'], dtype=object)

In [26]:
filter_stats_all = full.reset_index().groupby(['town','street_name','block','storey_range',
       'floor_area_sqm', 'flat_type','flat_model', 'lease_commence_date'])['index'].count().reset_index()
filter_stats_all.columns = ['town', 'street_name', 'block', 'storey_range', 'floor_area_sqm','flat_type',
       'flat_model', 'lease_commence_date', 'count']

In [27]:
filter_stats_2013 = full[full.year>=2013].reset_index().groupby(['town','street_name','block','storey_range',
       'floor_area_sqm', 'flat_type','flat_model', 'lease_commence_date'])['index'].count().reset_index()
filter_stats_2013.columns = ['town', 'street_name', 'block', 'storey_range', 'floor_area_sqm','flat_type',
       'flat_model', 'lease_commence_date', 'count_2013']

In [28]:
filter_stats = pd.merge(filter_stats_all,filter_stats_2013,on=['town','street_name','block','storey_range',
       'floor_area_sqm', 'flat_type','flat_model', 'lease_commence_date'],how='inner')

In [30]:
filter_stats = filter_stats[(filter_stats['count']>=5) & (filter_stats['count_2013']>2)].reset_index(drop=True)

In [31]:
unique_address = pd.read_excel('unique_address_geocode.xlsx')

In [32]:
full_filtered = pd.merge(filter_stats,full, on=['town','block','street_name','storey_range',
       'floor_area_sqm', 'flat_type','flat_model', 'lease_commence_date'],how='inner')
full_filtered = pd.merge(full_filtered,unique_address[['block','street_name','official_address','lat','lng',
                                            'station_no','station_name','station_color','station_distance_km',
                                            '1km_top50_school_count','1km_top50_schools']], 
                            how='left', on=['block','street_name'])

In [33]:
town_focus_tier1 = ['BISHAN', 'BUKIT MERAH','BUKIT TIMAH','CLEMENTI', 'KALLANG/WHAMPOA', 
                    'MARINE PARADE', 'QUEENSTOWN', 'TOA PAYOH']

In [34]:
full_filtered = full_filtered[(full_filtered['1km_top50_school_count']>=1)
                              & (full_filtered.station_distance_km<=1)
                              & (full_filtered.town.isin(town_focus_tier1))
                              & (full_filtered.floor_area_sqm>=85)
                              & (full_filtered.flat_type.isin(['4 room', '5 room']))
                             &(full_filtered.lease_commence_date>1990)]

In [37]:
full_stats = full_filtered.groupby(['town','block','street_name','storey_range',
       'floor_area_sqm', 'flat_type','flat_model', 'lease_commence_date','year']).agg({'psm':'median'}).reset_index()

In [38]:
full_stats.head()

,town,block,street_name,storey_range,floor_area_sqm,flat_type,flat_model,lease_commence_date,year,psm
0,BISHAN,209,BISHAN ST 23,10 TO 12,103.0,4 room,model a,1992,1996,4223.300971
1,BISHAN,209,BISHAN ST 23,10 TO 12,103.0,4 room,model a,1992,2000,3398.058252
2,BISHAN,209,BISHAN ST 23,10 TO 12,103.0,4 room,model a,1992,2002,3126.213592
3,BISHAN,209,BISHAN ST 23,10 TO 12,103.0,4 room,model a,1992,2005,2975.728155
4,BISHAN,209,BISHAN ST 23,10 TO 12,103.0,4 room,model a,1992,2008,3980.582524


In [39]:
full_stats.columns

Index(['town', 'block', 'street_name', 'storey_range', 'floor_area_sqm',
       'flat_type', 'flat_model', 'lease_commence_date', 'year', 'psm'],
      dtype='object')

In [43]:
full_stats['cagr'] = None
full_stats['cagr_yr'] = None

In [44]:
for i in range(1,full_stats.shape[0]):
    if (full_stats.loc[i-1,['town', 'block', 'street_name', 'storey_range', 'floor_area_sqm',
       'flat_type', 'flat_model', 'lease_commence_date']] == full_stats.loc[i,['town', 'block', 'street_name', 'storey_range', 'floor_area_sqm',
       'flat_type', 'flat_model', 'lease_commence_date']]).all():
        t = full_stats.loc[i,'year'] - full_stats.loc[i-1,'year']
        cagr = (full_stats.loc[i,'psm']/full_stats.loc[i-1,'psm'])**(1/t)-1
        full_stats.loc[i,'cagr'] = cagr
        full_stats.loc[i,'cagr_yr'] = t

In [55]:
cagr_stats = full_stats.dropna()
cagr_stats['cagr'] = cagr_stats['cagr'].astype(float)

/Users/ZhangYing/anaconda3/envs/re/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
cagr_stats['post_2013'] = cagr_stats['year'].apply(lambda x: True if x>=2013 else False)
cagr_stats = cagr_stats[cagr_stats['post_2013']==True]

/Users/ZhangYing/anaconda3/envs/re/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
post_2013_cagr = cagr_stats.groupby(['town','flat_type','lease_commence_date','cagr_yr'])['cagr'].median().reset_index()

In [71]:
post_2013_cagr.pivot_table(index=['town','flat_type','lease_commence_date'], columns='cagr_yr', values='cagr').reset_index().to_excel('cagr_summary.xlsx')

In [72]:
full.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,psm,year
0,2012-03,ANG MO KIO,2 room,172,ANG MO KIO AVE 4,06 TO 10,45.0,improved,1986,250000.0,5555.555556,2012
1,2012-03,ANG MO KIO,2 room,510,ANG MO KIO AVE 8,01 TO 05,44.0,improved,1980,265000.0,6022.727273,2012
2,2012-03,ANG MO KIO,3 room,610,ANG MO KIO AVE 4,06 TO 10,68.0,new generation,1980,315000.0,4632.352941,2012
3,2012-03,ANG MO KIO,3 room,474,ANG MO KIO AVE 10,01 TO 05,67.0,new generation,1984,320000.0,4776.119403,2012
4,2012-03,ANG MO KIO,3 room,604,ANG MO KIO AVE 5,06 TO 10,67.0,new generation,1980,321000.0,4791.044776,2012


In [83]:
full[(full.street_name.str.contains('BENDEMEER')) &(full.block.str.contains('39')) & (full.flat_type=='4 room')].sort_values('month')

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,psm,year
12437,2015-09,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,16 TO 18,89.0,model a,2012,725000.0,8146.067416,2015
22857,2016-04,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,25 TO 27,92.0,model a,2012,745000.0,8097.826087,2016
22854,2016-04,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,19 TO 21,92.0,model a,2012,715000.0,7771.739130,2016
24718,2016-05,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,16 TO 18,92.0,model a,2012,695000.0,7554.347826,2016
24717,2016-05,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,16 TO 18,92.0,model a,2012,685000.0,7445.652174,2016
24708,2016-05,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,01 TO 03,92.0,model a,2012,595000.0,6467.391304,2016
28212,2016-07,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,07 TO 09,92.0,model a,2012,668000.0,7260.869565,2016
35030,2016-11,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,22 TO 24,89.0,model a,2012,690000.0,7752.808989,2016
3204,2017-03,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,16 TO 18,92.0,model a,2012,680000.0,7391.304348,2017
3205,2017-03,KALLANG/WHAMPOA,4 room,39A,BENDEMEER RD,25 TO 27,92.0,model a,2012,701888.0,7629.217391,2017


In [84]:
full[(full.street_name.str.contains('BENDEMEER')) &(full.block.str.contains('38')) & (full.flat_type=='4 room')].sort_values('month')

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,psm,year
689,2015-01,KALLANG/WHAMPOA,4 room,38D,BENDEMEER RD,10 TO 12,92.0,model a,2012,650000.0,7065.217391,2015
3124,2015-03,KALLANG/WHAMPOA,4 room,38A,BENDEMEER RD,10 TO 12,92.0,model a,2012,680000.0,7391.304348,2015
4653,2015-04,KALLANG/WHAMPOA,4 room,38A,BENDEMEER RD,10 TO 12,92.0,model a,2012,675000.0,7336.956522,2015
6209,2015-05,KALLANG/WHAMPOA,4 room,38D,BENDEMEER RD,13 TO 15,92.0,model a,2012,695000.0,7554.347826,2015
6210,2015-05,KALLANG/WHAMPOA,4 room,38A,BENDEMEER RD,19 TO 21,92.0,model a,2012,730000.0,7934.782609,2015
7923,2015-06,KALLANG/WHAMPOA,4 room,38B,BENDEMEER RD,22 TO 24,92.0,model a,2012,728000.0,7913.043478,2015
10975,2015-08,KALLANG/WHAMPOA,4 room,38D,BENDEMEER RD,13 TO 15,89.0,model a,2012,696000.0,7820.224719,2015
12434,2015-09,KALLANG/WHAMPOA,4 room,38A,BENDEMEER RD,10 TO 12,92.0,model a,2012,688000.0,7478.260870,2015
12436,2015-09,KALLANG/WHAMPOA,4 room,38D,BENDEMEER RD,19 TO 21,92.0,model a,2012,720000.0,7826.086957,2015
14116,2015-10,KALLANG/WHAMPOA,4 room,38B,BENDEMEER RD,07 TO 09,89.0,model a,2012,627000.0,7044.943820,2015
